### Producer 1
This program loads all data from the climate_streaming.csv and randomly feeds data to the stream every 10 seconds. Additional information such as producer info and created date is attached.


In [ ]:
%pip install kafka

In [ ]:
import os
import pandas as pd
file_path = '/home/student/A2/Dataset/climate_streaming.csv'
print(os.path.exists(file_path))

# references
# https://stackoverflow.com/questions/78460124/cannot-access-files-local-path-after-mounting-local-path-to-jupyter-running-insi
# https://stackoverflow.com/questions/43574952/cant-load-data-onto-jupyter-docker
# streaming_data = pd.read_csv('/Dataset/climate_streaming.csv').to_dict(orient='records')

In [ ]:
from time import sleep
from json import dumps
from kafka3 import KafkaProducer
import random
import datetime as dt
from datetime import timedelta
import pymongo
from pymongo import MongoClient
import pandas as pd
import pprint

hostip = "10.192.104.132"  
client = MongoClient(hostip, 27017)
db = client["fit3182_assignment_db"]

#bro what is this i dont even rmb
climate_data_collection = db["climate_data"]

topic = 'PartB'

#referenced from week 10 lab, producer 1
def publish_message(producer_instance, topic_name, key, value):
    try:
        # kafka requires messages to be sent as bytes
        key_bytes = bytes(key, encoding='utf-8')
        value_bytes = bytes(value, encoding='utf-8')

        # producer instance is what you passed in
        # it works to send the message to the specified topic
        producer_instance.send(topic_name, key=key_bytes, value=value_bytes)
        # ensures message is actually sent to kafka before moving on to the next line
        producer_instance.flush()
        pprint.pprint('Message published successfully. Data: ' + str(value))
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))
        
def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=[f'{hostip}:9092'],
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer
    
# Function to modify streaming data and create a new date for each batch
def modify_streaming_data(streaming_data, latest_date):
    new_data = []
    current_date = latest_date
    for record in streaming_data:
        # Modify date for every 10 seconds of data
        current_date += timedelta(days=1)
        # Modify the record with the new date
        record['date'] = current_date.strftime('%Y-%m-%d')
        new_data.append(record)
    return new_data

if __name__ == '__main__':
   
    producer = connect_kafka_producer()
    
    print('Publishing records..')

    # Read data from climate_streaming.csv
    streaming_data = pd.read_csv('/home/student/A2/Dataset/climate_streaming.csv').to_dict(orient='records')

    # Get the latest date from MongoDB
    latest_date = climate_data_collection.find_one(sort=[('date', pymongo.DESCENDING)])
    print('Latest date from MongoDB:', latest_date['date'])

    modified_data = modify_streaming_data(streaming_data, dt.datetime.strptime(latest_date['date'], '%Y-%m-%d'))
    
    # Append additional information to each record and publish to Kafka
    for record in modified_data:
        # Append producer information and created date
        record['producer'] = 'Climate01'
        record['created_date'] = dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        # Publish the record to Kafka
        publish_message(producer, topic, 'parsed', dumps(record))
        sleep(10)

Publishing records..
Latest date from MongoDB: 2024-02-01
('Message published successfully. Data: {"latitude": -37.623, "longitude": '
 '149.323, "air_temperature_celcius": 19, "relative_humidity": 56.8, '
 '"windspeed_knots": 7.9, "max_wind_speed": 11.1, "precipitation ": " 0.00I", '
 '"GHI_w/m2": 154, "date": "2024-02-02", "producer": "Climate01", '
 '"created_date": "2024-05-27 05:17:36"}')
('Message published successfully. Data: {"latitude": -38.038, "longitude": '
 '142.986, "air_temperature_celcius": 15, "relative_humidity": 50.7, '
 '"windspeed_knots": 9.2, "max_wind_speed": 13.0, "precipitation ": " 0.02G", '
 '"GHI_w/m2": 128, "date": "2024-02-03", "producer": "Climate01", '
 '"created_date": "2024-05-27 05:17:46"}')
('Message published successfully. Data: {"latitude": -37.95, "longitude": '
 '142.366, "air_temperature_celcius": 16, "relative_humidity": 53.6, '
 '"windspeed_knots": 8.1, "max_wind_speed": 15.0, "precipitation ": " 0.00G", '
 '"GHI_w/m2": 133, "date": "2024-02-0

('Message published successfully. Data: {"latitude": -37.598, "longitude": '
 '149.29, "air_temperature_celcius": 18, "relative_humidity": 53.5, '
 '"windspeed_knots": 8.8, "max_wind_speed": 13.0, "precipitation ": " 0.00I", '
 '"GHI_w/m2": 150, "date": "2024-02-27", "producer": "Climate01", '
 '"created_date": "2024-05-27 05:21:46"}')
('Message published successfully. Data: {"latitude": -37.596, "longitude": '
 '149.319, "air_temperature_celcius": 19, "relative_humidity": 56.1, '
 '"windspeed_knots": 6.9, "max_wind_speed": 12.0, "precipitation ": " 0.00A", '
 '"GHI_w/m2": 155, "date": "2024-02-28", "producer": "Climate01", '
 '"created_date": "2024-05-27 05:21:56"}')
('Message published successfully. Data: {"latitude": -37.63, "longitude": '
 '149.232, "air_temperature_celcius": 18, "relative_humidity": 57.0, '
 '"windspeed_knots": 7.4, "max_wind_speed": 15.0, "precipitation ": " 0.00I", '
 '"GHI_w/m2": 145, "date": "2024-02-29", "producer": "Climate01", '
 '"created_date": "2024-05-2